# Questão 3 – Exemplo de Web Scraping
Este notebook demonstra duas rotinas de Web Scraping já utilizadas na empresa, adaptadas para salvar o resultado em CSV para posterior Mineração de Texto.
- **tabuaMare.py**: raspa tabela de marés e salva em JSON/CSV.
- **teste2.py**: outro exemplo de scraping (ex: horários de sol e lua) e salvamento.


In [1]:
import json
import pandas as pd


## 1. Rotina `tabuaMare.py`

In [2]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

webdriver_path = 'C:/chromedriver/chromedriver.exe'  

service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

def get_tide_data_for_month(year, month):
    url = f'https://surfguru.com.br/previsao/mare/60220/m?mes={month:02d}&ano={year}'
    
    driver.get(url)

    WebDriverWait(driver, 2).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'celula_dia'))
    )

    days = driver.find_elements(By.CLASS_NAME, 'celula_dia')

    month_data = []

    for day in days:
        day_data = day.text.strip()
        month_data.append(day_data)

    return month_data

year_data = {}

year = 25
for month in range(1, 13):
    print(f"Coletando dados para {month:02d}/{year}...")

    month_data = get_tide_data_for_month(year, month)
    
    year_data[f"{year}-{month:02d}"] = month_data

with open(f'tabua_mare_{year}.json', 'w', encoding='utf-8') as f:
    json.dump(year_data, f, ensure_ascii=False, indent=4)

driver.quit()

print(f"Dados de maré para o ano {year} foram exportados para 'tabua_mare_{year}.json'")


Coletando dados para 01/25...
Coletando dados para 02/25...
Coletando dados para 03/25...
Coletando dados para 04/25...
Coletando dados para 05/25...
Coletando dados para 06/25...
Coletando dados para 07/25...
Coletando dados para 08/25...
Coletando dados para 09/25...
Coletando dados para 10/25...
Coletando dados para 11/25...
Coletando dados para 12/25...
Dados de maré para o ano 25 foram exportados para 'tabua_mare_25.json'


### 1.1. Converter JSON de marés para CSV

In [2]:
import json
import pandas as pd

# Ajuste estes paths conforme sua estrutura
json_path = 'tabua_mare_25.json'
csv_path = 'tabua_mare_25.csv'

with open(json_path, 'r', encoding='utf-8') as jf:
    year_data = json.load(jf)

rows = []

# Iterador genérico de meses
if isinstance(year_data, dict):
    month_iter = year_data.items()
elif isinstance(year_data, list):
    # antiga versão: lista de dicts com keys 'mes' e 'dias'
    month_iter = ((m.get('mes'), m.get('dias', [])) for m in year_data)
else:
    raise ValueError(f"Formato inesperado: {type(year_data)}")

for mes, days in month_iter:
    # Se for dict (dia→info), transforma em lista de tuplas
    if isinstance(days, dict):
        days = [(dia, info) for dia, info in days.items()]
    # Agora days pode ser:
    #  - lista de tuplas (dia, info_dict)
    #  - lista de dicts (com chaves 'dia','marés')
    #  - **lista de strings** (formato Selenium)
    for entry in days:
        # caso tupla (dia, dict)
        if isinstance(entry, tuple):
            dia, info = entry
            tides = info.get('marés', [])
            for tide in tides:
                rows.append({
                    'mes':      mes,
                    'dia':      dia,
                    'hora':     tide.get('hora'),
                    'altura':   tide.get('altura')
                })

        # caso dict antigo
        elif isinstance(entry, dict):
            dia  = entry.get('dia') or entry.get('day')
            tides = entry.get('marés', [])
            for tide in tides:
                rows.append({
                    'mes':    mes,
                    'dia':    dia,
                    'hora':   tide.get('hora'),
                    'altura': tide.get('altura')
                })

        # **caso string do Selenium**
        elif isinstance(entry, str):
            lines = entry.splitlines()
            if not lines: 
                continue
            dia = lines[0].strip()
            # demais linhas: cada uma “HH:MM X.XXm”
            for tide_line in lines[1:]:
                parts = tide_line.strip().split()
                if len(parts) >= 2:
                    hora   = parts[0]
                    altura = parts[1]
                    rows.append({
                        'mes':    mes,
                        'dia':    dia,
                        'hora':   hora,
                        'altura': altura
                    })
        else:
            # tipos inesperados podem ser ignorados ou logados
            print(f"Aviso: tipo inesperado em days -> {type(entry)}")

# Grava o CSV final
df_mare = pd.DataFrame(rows)
df_mare.to_csv(csv_path, index=False, encoding='utf-8')
print(f'CSV de marés gerado em: {csv_path}')


CSV de marés gerado em: tabua_mare_25.csv


## 2. Rotina `teste2.py`

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

webdriver_path = 'C:/chromedriver/chromedriver.exe'

service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

url = 'https://surfguru.com.br/previsao/mare/60220/m?mes=05&ano=25'

driver.get(url)

data = {}
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'celula_dia'))
    )

    days = driver.find_elements(By.CLASS_NAME, 'celula_dia')

    for day in days:
        try:
            day_text = day.find_element(By.CLASS_NAME, 'linha_data_lua').text.strip()
        except Exception as e:
            print(f"Erro ao tentar encontrar o dia: {e}")
            continue

        day_number, day_name = day_text.split(" - ")
        tides = []

        mare_data = day.find_elements(By.CSS_SELECTOR, '.celula_mare, .celula_mare_baixa')
        for mare in mare_data:
            try:
                hour, height = mare.text.strip().split('h ')
                tides.append({
                    "hora": hour + "h",
                    "altura": height
                })
            except Exception as e:
                print(f"Erro ao tentar encontrar marés: {e}")
                continue
        try:
            sun_info = day.find_element(By.XPATH, ".//div[contains(text(), 'Nascer do Sol')]/following-sibling::div").text
            sun_rise, sun_set = sun_info.split(' - ')
        except:
            sun_rise, sun_set = "N/A", "N/A"

        try:
            moon_info = day.find_element(By.XPATH, ".//div[contains(text(), 'Nascer da Lua')]/following-sibling::div").text
            moon_rise, moon_set = moon_info.split(' - ')
        except:
            moon_rise, moon_set = "N/A", "N/A"

        data[day_number] = {
            "dia": day_name,
            "marés": tides,
            "sol": {
                "nasce": sun_rise,
                "se_põe": sun_set
            },
            "lua": {
                "nasce": moon_rise,
                "se_põe": moon_set
            }
        }

    data["marés"] = {
        "alta": {
            "altura": "1.9",
            "dias": ["08", "09", "10"]
        },
        "baixa": {
            "altura": "-0.3",
            "dia": "07"
        }
    }

finally:
    driver.quit()

with open("previsao_mare.json", "w", encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Dados exportados com sucesso!")


Dados exportados com sucesso!


### 2.1. Salvar resultado de `teste2.py` em CSV

In [4]:
# Suponha que o script produza um dicionário `data` com registros
# Ajuste a variável abaixo conforme o script
try:
    records = data  # `data` define lista de dicionários ou similar
    df2 = pd.DataFrame(records)
    df2.to_csv('resultado_teste2.csv', index=False, encoding='utf-8')
    print('CSV de teste2 gerado em: resultado_teste2.csv')
except NameError:
    print('Verifique se `data` está definido pelo teste2.py para CSV.')

CSV de teste2 gerado em: resultado_teste2.csv
